In [1]:
import os

os.chdir('../asr/')

In [2]:
from typing import Iterable, Tuple, List
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F
import sentencepiece
from omegaconf import OmegaConf
import pytorch_lightning as pl
import pandas as pd
from tqdm.auto import tqdm

from src.models import LASModel, CTCModel
from src.metrics import WER

NOTE! Installing ujson may make loading annotations faster.


In [3]:
!git diff --no-index conf/conformer_ctc.yaml conf/conformer_ctc_wide.yaml


diff --git a/conf/conformer_ctc.yaml b/conf/conformer_ctc_wide.yaml
index 10f8a87..6b58e13 100755
--- a/conf/conformer_ctc.yaml
+++ b/conf/conformer_ctc_wide.yaml
@@ -9,11 +9,11 @@ model:
     dropout: 0.0
     features_num: 64
     subsampling_stride: 4
-    d_model: 256
-    n_layers: 10
+    d_model: 320
+    n_layers: 8
     n_heads: 8
     ff_exp_factor: 2
-    kernel_size: 7
+    kernel_size: 15
     
 
   decoder:


In [4]:
def init_model(model: pl.LightningModule, ckpt_path: str) -> pl.LightningModule:
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt)
    model.eval()
    model.freeze()
    return model


def compute_wer(refs: Iterable[str], hyps: Iterable[str]) -> float:
    wer = WER()
    wer.update(refs, hyps)
    return wer.compute()[0].item()


class GreedyDecoderLAS:
    def __init__(self, model: LASModel, max_steps=20):
        self.model = model
        self.max_steps = max_steps

    def __call__(self, encoded: torch.Tensor) -> str:
        
        tokens = [self.model.decoder.tokenizer.bos_id()]

        for _ in range(self.max_steps):
            
            tokens_batch = torch.tensor(tokens).unsqueeze(0).to(self.model.device)
            att_mask = self.model.make_attention_mask(torch.tensor([tokens_batch.size(-1)])).to(self.model.device)
            
            distribution = self.model.decoder(
                encoded=encoded, encoded_pad_mask=None,
                target=tokens_batch, target_mask=att_mask, target_pad_mask=None
            )
        
            best_next_token = distribution[0, -1].argmax()
            
            if best_next_token == self.model.decoder.tokenizer.eos_id():
                break

            tokens.append(best_next_token.item())
        
        return self.model.decoder.tokenizer.decode(tokens)

## Single Model

In [6]:
dataset = 'test_opus/farfield/manifest.jsonl'

TOKENIZER_PATH = '../asr/data/checkpoints_and_tokenizer/tokenizer/'
LAS_CKPT_PATH = '../asr/data/checkpoints_and_tokenizer/conformer_las_2epochs.ckpt'

device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

### LAS

In [7]:
conf = OmegaConf.load("./conf/conformer_las.yaml")
conf.val_dataloader.dataset.manifest_name = dataset
conf.model.decoder.tokenizer_path = os.path.join(TOKENIZER_PATH, "bpe_1024_bos_eos.model")

conformer_las = init_model(
    model=LASModel(conf=conf),
    ckpt_path=LAS_CKPT_PATH
)
conformer_las = conformer_las.to(device)

In [8]:
print(sum(p.numel() for p in conformer_las.parameters()))


11121920


In [9]:
las_decoder = GreedyDecoderLAS(conformer_las)

refs, hyps_las = [], []

for batch in tqdm(conformer_las.val_dataloader()):

    features, features_len, targets, target_len = batch

    encoded, encoded_len = conformer_las(features.to(device), features_len.to(device))
    
    for i in range(features.shape[0]):

        encoder_states = encoded[
            [i],
            :encoded_len[i],
            :
        ]

        ref_tokens = targets[i, :target_len[i]].tolist()

        refs.append(
            conformer_las.decoder.tokenizer.decode(ref_tokens)
        )
        hyps_las.append(
            las_decoder(encoder_states)
        )

  0%|          | 0/60 [00:00<?, ?it/s]

In [11]:
compute_wer(refs, hyps_las)


0.42290276288986206

### CTC

In [10]:
CTC_WIDE_CKPT_PATH = '../asr/data/checkpoints_and_tokenizer/conformer_wide_7epochs_state_dict.ckpt'
CTC_DEEP_CKPT_PATH = '../asr/data/checkpoints_and_tokenizer/conformer_7epochs_state_dict.ckpt'

In [11]:
# TODO: load models, estimate WER

In [12]:
def decode_ctc_hyps(model: CTCModel) -> Tuple[List[str], List[str]]:
    return [], []

In [13]:
conf = OmegaConf.load("./conf/conformer_ctc.yaml")
conf.val_dataloader.dataset.manifest_name = dataset

conformer_ctc = init_model(
    model=CTCModel(conf=conf),
    ckpt_path=CTC_DEEP_CKPT_PATH
)

refs, hyps_ctc = decode_ctc_hyps(conformer_ctc)
print(sum(p.numel() for p in conformer_ctc.parameters()))

9989410


In [14]:
conf = OmegaConf.load("./conf/conformer_ctc_wide.yaml")
conf.val_dataloader.dataset.manifest_name = dataset

conformer_ctc = init_model(
    model=CTCModel(conf=conf),
    ckpt_path=CTC_WIDE_CKPT_PATH
)

refs, hyps_ctc = decode_ctc_hyps(conformer_ctc)
print(sum(p.numel() for p in conformer_ctc.parameters()))

12486114


## ROVER: Recognizer Output Voting Error Reduction — 5 points

* [A post-processing system to yield reduced word error rates: Recognizer Output Voting Error Reduction (ROVER)](https://ieeexplore.ieee.org/document/659110)
* [Improved ROVER using Language Model Information](https://www-tlp.limsi.fr/public/asr00_holger.pdf)

Alignment + Voting

![](images/rover_table.png)


In [ ]:
# ! pip install crowd-kit
from crowdkit.aggregation.texts import ROVER

In [27]:
# TODO: aggregate hypotheses, estimate WER


## MBR: Minimum Bayes Risk — 5 points

* [Minimum Bayes Risk Decoding and System Combination Based on a Recursion for Edit Distance](https://danielpovey.com/files/csl11_consensus.pdf)
* [mbr-decoding blog-post](https://suzyahyah.github.io/bayesian%20inference/machine%20translation/2022/02/15/mbr-decoding.html)
* [Combination of end-to-end and hybrid models for speech recognition](http://www.interspeech2020.org/uploadfile/pdf/Tue-1-8-4.pdf)

![](images/mbr_scheme.png)